In [22]:
import numpy as np
from matplotlib import pyplot as plt

In [23]:
# Boundary conditions
def apply_bcs(K_gl,F_gl,dof,value):
    phi = 1.0
    bignumber = 10.0**30
    K_gl[dof,dof] += bignumber*phi*phi
    F_gl[dof] += bignumber*float(value)*phi  

In [24]:
# Global index of local DOF
def gl_index(elem_index,local_dof):
    return elem_index + local_dof

In [25]:
# Global stiffness matrix and mass matrix
def GlobalSystem(E,L,A,n_elem,bc1,bc2):
    
    # Element stiffness matrix and element load vector
    h = L/n_elem
    k_elem = E*A/h * np.array([[1.0, -1.0], [-1.0, 1.0]])
    m_elem = rho*A*h/2 * np.array([[1.0,0.0],[0.0,1.0]])
    f_elem = np.array([0.0, 0.0])

    # Global stiffness matrix and load vector
    K_gl = np.zeros((n_elem+1, n_elem+1))
    F_gl = np.zeros((n_elem+1))
    M_gl = np.zeros((n_elem+1, n_elem+1))
    # Assembly of elements
    for i_el in range(n_elem):
        for i_loc in range(2):
            i_gl = gl_index(i_el, i_loc)
            F_gl[i_gl] += f_elem[i_loc]
            for j_loc in range(2):
                j_gl = gl_index(i_el, j_loc)
                K_gl[i_gl,j_gl] += k_elem[i_loc, j_loc]
                M_gl[i_gl,j_gl] += m_elem[i_loc, j_loc]

    # Boundary conditions
    apply_bcs(K_gl, F_gl, 0, bc1)
    # apply_bcs(K_gl, F_gl, n_elem, bc2)

    # Approximate solution
    # uh = np.linalg.solve(K_gl, F_gl)

    return K_gl, M_gl

In [26]:
# Newmark's method implicit
def newmark_imp(K, M, C, u, v, acel, p_next, dt, gamma, beta):
    #acel0 = np.matmul(np.linalg.inv(M),(p0 - C*v0 - K*u0))

    # Degrees of freedom
    dofs = K.shape[0]
    # Integration constants
    alpha = [
        1.0/(beta*dt**2.0),
        gamma/(beta*dt),
        1.0/(beta*dt),
        1.0/(2.0*beta)-1.0,
        gamma/beta-1.0,
        (gamma/beta-2.0)*dt/2.0,
        (1.0-gamma)*dt,
        gamma*dt
        ]

    # Effective stiffness matrix
    Keff = K + alpha[0]*M + alpha[1]*C
    
    # Keff inverse
    Kinv = np.linalg.inv(Keff)
    

    # Integration
    # Vector of effective forces at time
    pef = np.zeros((dofs))
    # Vector of displacements at time
    u_next = np.zeros((dofs))
    # Vector of accelerations  and velocities at time
    acel_next = np.zeros((dofs))
    v_next = np.zeros((dofs))

    # Vector of effective forces p
    term1 = alpha[0]*u + alpha[2]*v + alpha[3]*acel
    term2 = alpha[1]*u + alpha[4]*v + alpha[5]*acel
    pef_next = p_next + np.matmul(M,term1) + np.matmul(C,term2)
    # Vector of displacements u
    u_next = np.matmul(Kinv,pef_next)
    # Vector of accelerations acel and velocities v 
    acel_next = alpha[0]*(u_next-u) - alpha[2]*v - alpha[3]*acel
    v_next = v + alpha[6]*acel + alpha[7]*acel_next  
    
    return u_next,v_next,acel_next


# Newmark's method explicit
def newmark_exp(K, M, C, u, v, acel, p_next, dt, gamma, beta):

    # Degrees of freedom
    dofs = K.shape[0]
    
    up = np.zeros((dofs))
    vp = np.zeros((dofs))
    u_next = np.zeros((dofs))
    acel_next = np.zeros((dofs))
    v_next = np.zeros((dofs))

    # Predictors vectors
    up = u + dt*v + ((1.0/2.0 - beta)*dt**2)*acel
    vp = v + (1 - gamma)*dt*acel

    # Solution of the linear problem:
    term1 = M + gamma*dt/2*C + (beta*dt**2)*K
    Minv = np.linalg.inv(term1)
    term2 = p_next - np.matmul(K,up) - np.matmul(C,vp)
    acel_next = np.matmul(Minv,term2)

    # Correctors
    u_next = up + (beta*dt**2)*acel_next
    v_next = vp + gamma*dt*acel_next
    
    return u_next,v_next,acel_next

In [27]:
# Plot functions 

# Plot data in nodes
def plot(x,y,labelx,labely,title):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')
    plt.title(str(title))
    plt.xlabel(str(labelx))
    plt.ylabel(str(labely))
    plt.plot(x, y)
    plt.show()

# Plot data in element
def plot_interior(x0,xf,n_elem,func,labelx,labely,title):
    h = float(xf-x0)/n_elem
    x = np.array([[x0+h*el,x0+h*(el+1)] for el in range(n_elem)])
    x = x.flatten()
    y = np.array([[func[el],func[el]] for el in range(n_elem)])
    y = y.flatten()
    plot(x,y,labelx,labely,title)

#Plot data in time
def plot_time(n_steps,func,labelx,labely,title):
    x = np.linspace(0, n_steps, n_steps)
    y = func
    plot(x,y,labelx,labely,title)

# Plot energy 
def plot_energy(n_steps,E_kin,E_pot,labelx,labely,title):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')
    plt.title(str(title))
    plt.xlabel(str(labelx))
    plt.ylabel(str(labely))
    x = np.linspace(0, n_steps, n_steps)
    plt.plot(x, E_kin, label='E_kin')
    plt.plot(x, E_pot, label='E_pot')
    plt.legend()
    plt.show()
def plot_energytotal(n_steps,E_kin,E_pot,E_tot,labelx,labely,title):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')
    plt.title(str(title))
    plt.xlabel(str(labelx))
    plt.ylabel(str(labely))
    x = np.linspace(0, n_steps, n_steps)
    plt.plot(x, E_kin, label='E_kin')
    plt.plot(x, E_pot, label='E_pot')
    plt.plot(x, E_tot, label='E_tot')
    plt.legend()
    plt.show()    

In [28]:
# Energy balance
def energy(K,M,u,v):
    # Kinetic energy
    E_kin = 1.0/2.0*np.dot(np.matmul(M,v),v)
    # Potential energy
    E_pot = 1.0/2.0*np.dot(np.matmul(K,u),u)
    # Total 
    E_tot = E_kin + E_pot
    return E_kin, E_pot, E_tot

In [35]:
# Mesh

# Connectivity 
connect = [[0,1],[1,2],[0,3],[2],[3]]

1

In [33]:
# Input parameters

# Material parameters
E = 275.0*10**3 #(Pa)
L = 0.2 #(m)
A = 0.001 #(unit area)
Gc = 100.0 #(N/m)
stress_c = 300.0*10**3 #(Pa)
rho = 75.0 #(kg/m3)
delta_c = (2.0*Gc)/stress_c

# Mesh
n_elem = 2
h = L/n_elem
dofs = n_elem + 1
#Element index
elem_index = np.linspace(0,n_elem-1, n_elem)
# Element type convenction: 0(line element)/1(cohesive element)
elem_type = np.zeros(n_elem)
# Connectivity
connect = np.zeros((n_elem,2))
for i in range(n_elem):
    connect[i,0] = i 
    connect[i,1] = i + 1
        
# Time steps
n_steps = 10

# Boundary conditions
bc1 = 1
# Applied strain rate
strain_rate = 10.0**4 #(s-1)

# Initial displacement 
u0 = np.zeros((dofs))
# Initial velocity (v0): velocity profile (vel) is a function v(x)
vel = strain_rate*L
n_points = dofs
l = np.linspace(0, L, n_points)
v0 = np.array([vel/L*x for x in l])
v0 = np.round(v0, 8)

# Initial acceleration (acel0)
acel0 = np.zeros((dofs))
# Load (p)
p = np.zeros((n_steps+1,dofs))

# Time integration
dt = 10**-4 #(s)
gamma = 0.5
beta = 0.0 
C = np.zeros((dofs,dofs))  

dt_crit = h/((E/rho)**0.5)
print(dt_crit)


0.0006666666666666666
0.0016514456476895412


In [ ]:
def cohesive_law(stress_c,delta_c,delta):
    stress = stress_c*(1.0 - delta/delta_c)
    

In [37]:
# Global algorithm 

# Get K_gl and M_gl
K,M = GlobalSystem(E,L,A,n_elem,bc1,0)

# Get u,v,a,stain and stress 
u = u0
v = v0
acel = acel0

stress_evolution = np.zeros((n_steps, n_elem))
E_kin = np.zeros((n_steps))
E_pot = np.zeros((n_steps))
E_tot = np.zeros((n_steps))

up_n_elem = n_elem
up_dofs = dofs

for n in range(n_steps):

    # plot(l,v,"x","v","Velocity")
    # plot(l,u,"x","u","Displacement")
    u,v,acel = newmark_exp(K,M,C,u,v,acel,p[n+1],dt,gamma,beta)
    print(u)
    
    # Post process returns energy, strain and stress
    E_kin[n],E_pot[n],E_tot[n] = energy(K,M,u,v)
    strain = np.zeros(n_elem)
    stress = np.zeros(n_elem)

    for elem in range(up_n_elem):
        if elem_type[elem] == 0:
            strain[elem] = (u[elem+1] - u[elem])/h
            stress[elem] = E*strain[elem]
        #Cohesive law
        delta[elem] = 
        stress[elem] = cohesive_law(stress_c,delta_c,delta)

        # Fragmentation process 

        # Check limit stress
        if elem > 0:
            average_stress = (stress[elem] + stress[elem-1])/2.0

            if average_stress > stress_c:

                # Creation of new element
                # Update number of elements and dofs
                up_n_elem = up_n_elem + 1
                up_dofs = up_dofs + 1
                # Update element index
                elem_index = np.linspace(0,up_n_elem-1, up_n_elem)
                # Update element type: 0(line element)/1(cohesive element)
                elem_type = np.insert(elem_type, elem, 1, 0)
                # Update connectivity
                connect = np.zeros((up_n_elem,2))
                for i in range(up_n_elem):
                    connect[i,0] = i 
                    connect[i,1] = i + 1
                # Update stiffness and mass matrix
                # Size of the matrix (new columns and lines filled with 0)
                new_line = np.zeros(up_dofs-1)
                new_colum =np.zeros(up_dofs)
                K = np.insert(K, elem, new_line, 0)
                M = np.insert(M, elem, new_line, 0)
                K = np.append(K, elem, new_colum, 1)
                M = np.append(M, elem, new_colum, 1)
                # Update u, v and acel
                # Size, new line filled with zero
                u = np.insert(u, elem, 0, 0)
                v = np.insert(u, elem, 0, 0)
                acel = np.insert(u, elem, 0, 0)
                


                
                


    


# Variation of energy [n_steps, E_kin, E_pot] return the variation of energy between time steps
# vEkin = np.zeros((n_steps))
# vEpot = np.zeros((n_steps))
# vEtot = np.zeros((n_steps))

# for n in range(n_steps-1):
#     vEkin[n+1] = E_kin[n+1] - E_kin[n]
#     vEpot[n+1] = E_pot[n+1] - E_pot[n]
#     vEtot[n+1] = vEkin[n+1] + vEpot[n+1]

# Plots of energy values
# plot_energy(n_steps,E_kin,E_pot,"time_step","Energy","Energy balance")
# Plots of variation energy values
# plot_energytotal(n_steps,vEkin,vEpot,vEtot,"time_step","Energy","Energy balance")


[0.  0.1 0.2]
[0.00073333 0.2        0.39926667]
[-1.95555556e+21  3.00000000e-01  5.97072044e-01]
[ 5.21481481e+45 -7.17037037e+18  7.92698894e-01]
[-1.39061728e+70  1.91209877e+43 -5.25827160e+16]
[ 3.70831276e+94 -5.09893004e+67  1.40220576e+41]
[-9.88883402e+118  1.35971468e+092 -3.73921536e+065]
[ 2.63702241e+143 -3.62590581e+116  9.97124097e+089]
[-7.03205975e+167  9.66908215e+140 -2.65899759e+114]
[ 1.87521593e+192 -2.57842191e+165  7.09066024e+138]
